In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train_path = "data/raw/test.csv"
test_path = "data/raw/test.csv"

# Read the train and test datasets
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

# Merge the datasets
df = pd.concat([train_df, test_df], axis=0, ignore_index=True)

# Display the first 5 lines
df.head()


,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,1,22,Female,25,14,4,27,Basic,Monthly,598,9,1
1,2,41,Female,28,28,7,13,Standard,Monthly,584,20,0
2,3,47,Male,27,10,2,29,Premium,Annual,757,21,0
3,4,35,Male,9,12,5,17,Premium,Quarterly,232,18,0
4,5,53,Female,58,24,9,2,Standard,Annual,533,18,0


In [3]:
""" 
Q- now we need to check the table describtion of the data 

"""
df.describe()

,CustomerID,Age,Tenure,Usage Frequency,Support Calls,Payment Delay,Total Spend,Last Interaction,Churn
count,128748.000000,128748.000000,128748.000000,128748.000000,128748.000000,128748.000000,128748.000000,128748.000000,128748.000000
mean,32187.500000,41.970982,31.994827,15.080234,5.400690,17.133952,541.023379,15.498850,0.473685
std,18583.245281,13.924857,17.098168,8.816436,3.113993,8.852177,260.873796,8.638403,0.499309
min,1.000000,18.000000,1.000000,1.000000,0.000000,0.000000,100.000000,1.000000,0.000000
25%,16094.000000,30.000000,18.000000,7.000000,3.000000,10.000000,313.000000,8.000000,0.000000
50%,32187.500000,42.000000,33.000000,15.000000,6.000000,19.000000,534.000000,15.000000,0.000000
75%,48281.000000,54.000000,47.000000,23.000000,8.000000,25.000000,768.000000,23.000000,1.000000
max,64374.000000,65.000000,60.000000,30.000000,10.000000,30.000000,1000.000000,30.000000,1.000000


In [4]:
""" 
Q- check some information about the data ? 
"""
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128748 entries, 0 to 128747
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   CustomerID         128748 non-null  int64 
 1   Age                128748 non-null  int64 
 2   Gender             128748 non-null  object
 3   Tenure             128748 non-null  int64 
 4   Usage Frequency    128748 non-null  int64 
 5   Support Calls      128748 non-null  int64 
 6   Payment Delay      128748 non-null  int64 
 7   Subscription Type  128748 non-null  object
 8   Contract Length    128748 non-null  object
 9   Total Spend        128748 non-null  int64 
 10  Last Interaction   128748 non-null  int64 
 11  Churn              128748 non-null  int64 
dtypes: int64(9), object(3)
memory usage: 11.8+ MB


In [5]:
""" 
Q- let's Rename the columns and remove the empty space oky 

"""

df.columns = [col.lower().replace(' ','_') for col in df.columns]
df.columns


Index(['customerid', 'age', 'gender', 'tenure', 'usage_frequency',
       'support_calls', 'payment_delay', 'subscription_type',
       'contract_length', 'total_spend', 'last_interaction', 'churn'],
      dtype='object')

In [6]:
""" 
Q- let's remove the columns with unncessry information 
A- it's the CustomerID column oky 

"""
df.drop(columns='customerid',inplace=True)
df.shape

(128748, 11)

In [7]:
from typing import Callable, List
import pandas as pd

def compute_null_percentage(column: pd.Series) -> float:
    """
    Compute the percentage of null values in a given column.

    Args:
        column (pd.Series): The input column to analyze.

    Returns:
        float: The percentage of null values in the column.
    """
    null_count: int = column.isnull().sum()
    total_count: int = len(column)

    return (null_count / total_count) * 100 if total_count > 0 else 0.0

def analyze_null_percentages(df: pd.DataFrame) -> pd.Series:
    """
    Analyze null percentages for all columns in a DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame to analyze.

    Returns:
        pd.Series: A series of null percentages for each column.
    """
    return df.isnull().mean() * 100

# Call the analyze_null_percentages function
null_percentages = analyze_null_percentages(df)

# Print the results
for column, percentage in null_percentages.items():
    print(f"{column}: {percentage:.2f}% null values")

age: 0.00% null values
gender: 0.00% null values
tenure: 0.00% null values
usage_frequency: 0.00% null values
support_calls: 0.00% null values
payment_delay: 0.00% null values
subscription_type: 0.00% null values
contract_length: 0.00% null values
total_spend: 0.00% null values
last_interaction: 0.00% null values
churn: 0.00% null values


In [8]:
df.dropna(inplace=True) # Removing the single missing value row

In [9]:
df.isnull().sum()

age                  0
gender               0
tenure               0
usage_frequency      0
support_calls        0
payment_delay        0
subscription_type    0
contract_length      0
total_spend          0
last_interaction     0
churn                0
dtype: int64

In [10]:
""" 
Q- next we have some categorical features that we need to encode.
Q- before starting the ml modeling oky 

Q- the featuers are 
- gender 
-subscription_type
-contract_length
Q- so we use the OneHotEncoder to encode the categorical features
Q- how to logic is done ? 
- firtst we need to call the packge to do this wich is "OneHotEncoder"
- then we make an instance of it as encoder varible 
- then we fit the encoder to the data
Q- so give the full code here without removing the """ """ comments oky 

"""
from sklearn.preprocessing import OneHotEncoder

# List of categorical columns to encode
categorical_features = ['gender', 'subscription_type', 'contract_length']

# Create an instance of OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the data and transform the categorical columns
encoded_features = encoder.fit_transform(df[categorical_features])

# Create a DataFrame from the encoded features
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_features))

# Drop the original categorical columns from the DataFrame
df.drop(columns=categorical_features, inplace=True)

# Concatenate the encoded features with the original DataFrame
df = pd.concat([df, encoded_df], axis=1)

# Display the first 5 lines of the updated DataFrame
df.head()

,age,tenure,usage_frequency,support_calls,payment_delay,total_spend,last_interaction,churn,gender_Female,gender_Male,subscription_type_Basic,subscription_type_Premium,subscription_type_Standard,contract_length_Annual,contract_length_Monthly,contract_length_Quarterly
0,22,25,14,4,27,598,9,1,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,41,28,28,7,13,584,20,0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,47,27,10,2,29,757,21,0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
3,35,9,12,5,17,232,18,0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
4,53,58,24,9,2,533,18,0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [11]:
df.dropna(inplace=True)

# Verify that all null values have been dropped
print(df.isnull().sum())

age                           0
tenure                        0
usage_frequency               0
support_calls                 0
payment_delay                 0
total_spend                   0
last_interaction              0
churn                         0
gender_Female                 0
gender_Male                   0
subscription_type_Basic       0
subscription_type_Premium     0
subscription_type_Standard    0
contract_length_Annual        0
contract_length_Monthly       0
contract_length_Quarterly     0
dtype: int64


In [12]:
""" 
Q- now we have clean the data , encoding it 
Q- it's time to prepare the train and test data
Q- first think we need to import the needed libraries like train_test_split from sklearn.model_selection
Q- and StandardScaler from sklearn.preprocessing
Q- then due this problem is a supervised learning problem and the target is "churn" we need to remove it from the data
Q- and then we need to split the data into train and test data
Q- and then we need to scale the data
"""
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X=df.drop("churn",axis=1)
y=df["churn"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# identify numerical columns 
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
# intialize the scaler
scaler = StandardScaler()
# fit and transform the numerical columns
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
# transform the test set 
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

# print the result 
print("Training set shape :" , X_train.shape)
print("Test set shape :" , X_test.shape)

Training set shape : (102998, 15)
Test set shape : (25750, 15)


In [14]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from tabulate import tabulate

def train_and_validate_model(X_train, X_test, y_train, y_test, model=LogisticRegression()):
    """
    Train a model and validate its performance using various metrics.

    Input:
    - X_train: Training features
    - X_test: Testing features
    - y_train: Training labels
    - y_test: Testing labels
    - model: Machine learning model (default: LogisticRegression)

    Output:
    - accuracy: Accuracy score
    - precision: Precision score
    - recall: Recall score
    - f1: F1 score
    - roc_auc: ROC AUC score
    """
    # Train the model
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]

    # Calculate performance metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    return accuracy, precision, recall, f1, roc_auc
"""
Q- now just print the result after calling the method 
Q- give the code to it 
"""
result = train_and_validate_model(X_train, X_test, y_train, y_test)
metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC AUC']
for metric, value in zip(metrics, result):
    print(f"{metric}: {value:.4f}")

Accuracy: 0.8282
Precision: 0.8093
Recall: 0.8326
F1 Score: 0.8207
ROC AUC: 0.9039


In [16]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import mlflow
import mlflow.sklearn

def train_and_validate_model(X_train, X_test, y_train, y_test, model=LogisticRegression()):
    with mlflow.start_run():
        # Log model parameters
        mlflow.log_param("model", "LogisticRegression")
        mlflow.log_param("test_size", 0.2)
        mlflow.log_param("random_state", 42)
        
        # Train the model
        model.fit(X_train, y_train)

        # Make predictions
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]

        # Calculate performance metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_pred_proba)

        # Log metrics
        metrics = ['accuracy', 'precision', 'recall', 'f1_score', 'roc_auc']
        for metric, value in zip(metrics, [accuracy, precision, recall, f1, roc_auc]):
            mlflow.log_metric(metric, value)
            print(f"{metric.capitalize()}: {value:.4f}")

        # Log the model
        mlflow.sklearn.log_model(model, "logistic_regression_model")

        return accuracy, precision, recall, f1, roc_auc

# Call the function
result = train_and_validate_model(X_train, X_test, y_train, y_test)


Accuracy: 0.8282
Precision: 0.8093
Recall: 0.8326
F1_score: 0.8207
Roc_auc: 0.9039


2024/10/16 16:33:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [17]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, roc_auc_score)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import mlflow
import mlflow.sklearn

def train_and_validate_model(X_train, X_test, y_train, y_test, model):
    with mlflow.start_run():
        mlflow.log_param("model", model.__class__.__name__)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        if hasattr(model, "predict_proba"):
            y_pred_proba = model.predict_proba(X_test)[:, 1]
        else:
            y_pred_proba = y_pred
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_pred_proba) if hasattr(model, "predict_proba") else None
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)
        if roc_auc is not None:
            mlflow.log_metric("roc_auc", roc_auc)
        print(f"Model: {model.__class__.__name__}")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")
        if roc_auc is not None:
            print(f"ROC AUC: {roc_auc:.4f}")
        mlflow.sklearn.log_model(model, model.__class__.__name__)
        return accuracy, precision, recall, f1, roc_auc

knn = KNeighborsClassifier(n_neighbors=5)
result_knn = train_and_validate_model(X_train, X_test, y_train, y_test, knn)


Model: KNeighborsClassifier
Accuracy: 0.9339
Precision: 0.9095
Recall: 0.9551
F1 Score: 0.9318
ROC AUC: 0.9787


2024/10/16 16:33:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [18]:
from sklearn.tree import DecisionTreeClassifier
def train_and_validate_decision_tree(X_train,X_test,y_train,y_test,max_depth=None,random_state=None):
    """ 
    Train a Decision Tree model and validate its performance using various metrics.
    The function also log the model , parameters , and metrics to the MLflow.
     Input:
    - X_train: Training features
    - X_test: Testing features
    - y_train: Training labels
    - y_test: Testing labels
    - max_depth: Maximum depth of the decision tree (default: None)
    - random_state: Random state for reproducibility (default: 42)

    Output:
    - accuracy: Accuracy score
    - precision: Precision score
    - recall: Recall score
    - f1: F1 score
    - roc_auc: ROC AUC score
    """
    with mlflow.start_run():
        model =DecisionTreeClassifier(max_depth=max_depth,random_state=random_state)
        model.fit(X_train,y_train)

        # log the model parameters
        mlflow.log_param("model","DecisionTreeClassifier")
        mlflow.log_param("max_depth",max_depth)
        mlflow.log_param("random_state",random_state)

        # make predictions 
        y_pred=model.predict(X_test)
        y_pred_proba=model.predict_proba(X_test)[:,1]
        
        # Calculate evaluation metrics
        accuracy=accuracy_score(y_test,y_pred)
        precision=precision_score(y_test,y_pred)
        recall=recall_score(y_test,y_pred)
        f1=f1_score(y_test,y_pred)
        roc_auc= roc_auc_score(y_test,y_pred_proba)

        # log the metrics
        metrics=['accuracy','precision','recall','f1','roc_auc']
        for metric , value in zip(metrics,[accuracy,precision,recall,f1,roc_auc]):
            mlflow.log_metric(metric,value)
            print(f"{metric.capitalize()}:{value:.4f}")
        # log the model
        mlflow.sklearn.log_model(model,"decision_tree_model")
        return accuracy,precision,recall,f1,roc_auc
results=train_and_validate_decision_tree(X_train,X_test,y_train,y_test,max_depth=5,random_state=42)


Accuracy:0.9551
Precision:0.9277
Recall:0.9816
F1:0.9539
Roc_auc:0.9942


2024/10/16 17:10:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
